In [ ]:
# study domain size (demo)
import numpy
import matplotlib.pyplot as plt
from pyscf import gto
from scipy.io import savemat
import plotly.graph_objects as go
from utils.cheby import cheby
from utils.perispecdiff import perispecdiff

# basis
mol_h2o = gto.M(
    verbose = 0,
    atom = '''
    o    0    0.       0.
    h    0    -0.757   0.587
    h    0    0.757    0.587''',
    basis = 'ccpvqz') 
    # basis = 'ccpvtz') 
    # basis = 'ccpvdz') 

nd = 24
def pyscffunc(x, y, z, mol):
  return numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))

def mysphere(rho,p):
  nu = p
  nv = 2*p
  lam = numpy.linspace(0,nv-1,nv)*2*numpy.pi/nv
  gxt, gwt, D = cheby(nu,1)
  theta = numpy.arccos(gxt)
  [v, u] = numpy.meshgrid(lam,theta,indexing='ij')
  x = rho*numpy.sin(u)*numpy.cos(v)
  y = rho*numpy.sin(u)*numpy.sin(v)
  z = rho*numpy.cos(u)
  sw = rho**2*2*numpy.pi/nv*numpy.matlib.repmat(gwt,1,nv).transpose()
  return x,y,z,sw

p = 10
rho = 1
x,y,z,sw = mysphere(rho,p)

# scatter3d
trace = go.Scatter3d(x=x.flatten(),y=y.flatten(),z=z.flatten(),mode='markers',
  marker=dict(size=4,color='blue',opacity=1, 
  symbol='circle',line=dict(color='black',width=1))
)
fig = go.Figure(data=[trace])
fig.show()


In [ ]:
# do the work to figure out the domain size (for H2O)
tol_list = numpy.array([1.0e-2, 1.0e-3, 1.0e-4, 1.0e-5])
basis_list = ['ccpvdz', 
             'ccpvtz',
             'ccpvqz']
nd_list = numpy.array([24, 58, 115]) # does not seem to be used here
rho_list = numpy.zeros((tol_list.size,nd_list.size))

# loop over tol
for i in range(tol_list.size):
  # loop over d,t,qz
  for j in range(nd_list.size):
    mol_h2o = gto.M(
      verbose = 0,
      atom = '''
      o    0    0.       0.
      h    0    -0.757   0.587
      h    0    0.757    0.587''',
      basis = basis_list[j])
    nd = nd_list[j]
    # basis to be integrated on sphere surface
    def pyscffunc(x, y, z, mol):
      return numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
    func = lambda x, y, z: pyscffunc(x, y, z, mol_h2o)
    # compute sphere surface integral
    p = 10
    rhoall = numpy.arange(1, 31)
    basisints = numpy.zeros(rhoall.size)
    rho = 1/3
    x,y,z,sw = mysphere(rho,p)
    basisints0 = numpy.sqrt(numpy.sum( \
                          sw.flatten()[:,numpy.newaxis] * func(x.flatten(),y.flatten(),z.flatten())**2,axis=0))
    # breakpoint()
    for k in range(rhoall.size):
      rho = rhoall[k]
      x,y,z,sw = mysphere(rho,p)
      basisints[k] = numpy.max(numpy.sqrt(numpy.sum( \
                        sw.flatten()[:,numpy.newaxis] * func(x.flatten(),y.flatten(),z.flatten())**2,axis=0)) \
                        /basisints0)

    idx_h2o = numpy.sum(basisints > tol_list[i])
    # basisints[idx_h2o], rhoall[idx_h2o]
    rho_list[i,j] = rhoall[idx_h2o]

rho_list  

In [ ]:
# do the work to figure out the domain size (for HF)
tol_list = numpy.array([1.0e-2, 1.0e-3, 1.0e-4, 1.0e-5])
basis_list = ['ccpvdz', 
             'ccpvtz',
             'ccpvqz']
nd_list = numpy.array([19, 44, 85])
rho_list = numpy.zeros((tol_list.size,nd_list.size))

# loop over tol
for i in range(tol_list.size):
  # loop over d,t,qz
  for j in range(nd_list.size):
    mol_hf = gto.M(
      verbose = 0,
      atom = '''
      h    0    0.       0.
      f    0    0.       0.92''',
      basis = basis_list[j])
    nd = nd_list[j]
    # basis to be integrated on sphere surface
    def pyscffunc(x, y, z, mol):
      return numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
    func = lambda x, y, z: pyscffunc(x, y, z, mol_hf)
    # compute sphere surface integral
    p = 10
    rhoall = numpy.arange(1, 31)
    basisints = numpy.zeros(rhoall.size)
    rho = 1/3
    x,y,z,sw = mysphere(rho,p)
    basisints0 = numpy.sqrt(numpy.sum( \
                          sw.flatten()[:,numpy.newaxis] * func(x.flatten(),y.flatten(),z.flatten())**2,axis=0))
    # breakpoint()
    for k in range(rhoall.size):
      rho = rhoall[k]
      x,y,z,sw = mysphere(rho,p)
      basisints[k] = numpy.max(numpy.sqrt(numpy.sum( \
                        sw.flatten()[:,numpy.newaxis] * func(x.flatten(),y.flatten(),z.flatten())**2,axis=0)) \
                        /basisints0)

    idx_hf = numpy.sum(basisints > tol_list[i])
    rho_list[i,j] = rhoall[idx_hf]

rho_list  

In [ ]:
# do the work to figure out the domain size (for H2)
tol_list = numpy.array([1.0e-2, 1.0e-3, 1.0e-4, 1.0e-5])
basis_list = ['ccpvdz', 
             'ccpvtz',
             'ccpvqz']
nd_list = numpy.array([10, 28, 60])
rho_list = numpy.zeros((tol_list.size,nd_list.size))

# loop over tol
for i in range(tol_list.size):
  # loop over d,t,qz
  for j in range(nd_list.size):
    mol_h2 = gto.M(
      verbose = 0,
      atom = '''
      h    0    0.       0.
      h    0    0.       0.74''',
      basis = basis_list[j])
    nd = nd_list[j]
    # basis to be integrated on sphere surface
    def pyscffunc(x, y, z, mol):
      return numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
    func = lambda x, y, z: pyscffunc(x, y, z, mol_h2)
    # compute sphere surface integral
    p = 10
    rhoall = numpy.arange(1, 31)
    basisints = numpy.zeros(rhoall.size)
    rho = 1/3
    x,y,z,sw = mysphere(rho,p)
    basisints0 = numpy.sqrt(numpy.sum( \
                          sw.flatten()[:,numpy.newaxis] * func(x.flatten(),y.flatten(),z.flatten())**2,axis=0))
    # breakpoint()
    for k in range(rhoall.size):
      rho = rhoall[k]
      x,y,z,sw = mysphere(rho,p)
      basisints[k] = numpy.max(numpy.sqrt(numpy.sum( \
                        sw.flatten()[:,numpy.newaxis] * func(x.flatten(),y.flatten(),z.flatten())**2,axis=0)) \
                        /basisints0)

    idx_h2 = numpy.sum(basisints > tol_list[i])
    rho_list[i,j] = rhoall[idx_h2]

rho_list  

In [ ]:
# do the work to figure out the domain size (for CH4)
tol_list = numpy.array([1.0e-2, 1.0e-3, 1.0e-4, 1.0e-5])
basis_list = ['ccpvdz', 
             'ccpvtz',
             'ccpvqz']
nd_list = numpy.array([34, 86, 175])
rho_list = numpy.zeros((tol_list.size,nd_list.size))

# loop over tol
for i in range(tol_list.size):
  # loop over d,t,qz
  for j in range(nd_list.size):
    mol_ch4 = gto.M(
      verbose = 0,
      atom = '''
      C 0.000 0.000 0.000
      H 0.630 0.630 0.630
      H -0.630 -0.630 0.630
      H 0.630 -0.630 -0.630
      H -0.630 0.630 -0.630''',
      basis = basis_list[j])
    nd = nd_list[j]
    # basis to be integrated on sphere surface
    def pyscffunc(x, y, z, mol):
      return numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
    func = lambda x, y, z: pyscffunc(x, y, z, mol_ch4)
    # compute sphere surface integral
    p = 10
    rhoall = numpy.arange(1, 31)
    basisints = numpy.zeros(rhoall.size)
    rho = 1/3
    x,y,z,sw = mysphere(rho,p)
    basisints0 = numpy.sqrt(numpy.sum( \
                          sw.flatten()[:,numpy.newaxis] * func(x.flatten(),y.flatten(),z.flatten())**2,axis=0))
    # breakpoint()
    for k in range(rhoall.size):
      rho = rhoall[k]
      x,y,z,sw = mysphere(rho,p)
      basisints[k] = numpy.max(numpy.sqrt(numpy.sum( \
                        sw.flatten()[:,numpy.newaxis] * func(x.flatten(),y.flatten(),z.flatten())**2,axis=0)) \
                        /basisints0)

    idx_ch4 = numpy.sum(basisints > tol_list[i])
    rho_list[i,j] = rhoall[idx_ch4]

rho_list  

In [ ]:
# do the work to figure out the domain size (for CH4)
tol_list = numpy.array([1.0e-2, 1.0e-3, 1.0e-4, 1.0e-5])
basis_list = ['ccpvdz', 
             'ccpvtz',
             'ccpvqz']
nd_list = numpy.array([48, 116, 230])
rho_list = numpy.zeros((tol_list.size,nd_list.size))

# loop over tol
for i in range(tol_list.size):
  # loop over d,t,qz
  for j in range(nd_list.size):
    mol_c2h4 = gto.M(
      verbose = 0,
      atom = '''
      C 0.000 0.000 0.000
      C 1.333 0.000 0.000
      H -0.176 1.109 0.000
      H -0.176 -1.109 0.000
      H 1.509 1.109 0.000
      H 1.509 -1.109 0.000''',
      basis = basis_list[j])
    nd = nd_list[j]
    # basis to be integrated on sphere surface
    def pyscffunc(x, y, z, mol):
      return numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
    func = lambda x, y, z: pyscffunc(x, y, z, mol_c2h4)
    # compute sphere surface integral
    p = 10
    rhoall = numpy.arange(1, 31)
    basisints = numpy.zeros(rhoall.size)
    rho = 1/3
    x,y,z,sw = mysphere(rho,p)
    basisints0 = numpy.sqrt(numpy.sum( \
                          sw.flatten()[:,numpy.newaxis] * func(x.flatten(),y.flatten(),z.flatten())**2,axis=0))
    # breakpoint()
    for k in range(rhoall.size):
      rho = rhoall[k]
      x,y,z,sw = mysphere(rho,p)
      basisints[k] = numpy.max(numpy.sqrt(numpy.sum( \
                        sw.flatten()[:,numpy.newaxis] * func(x.flatten(),y.flatten(),z.flatten())**2,axis=0)) \
                        /basisints0)

    idx_c2h4 = numpy.sum(basisints > tol_list[i])
    rho_list[i,j] = rhoall[idx_c2h4]

rho_list  